# Adding Visual Features

### Step 1: Coordinate Extraction

In [1]:
# Ines's code takes PDF and yields pdf_word_list, coordinate_map

### Step 2: PDF to HTML Conversion

Use Adobe Acrobat (or other program of your choice) to convert PDF -> HTML with structure.

### Step 3: HTML Parsing

In [ ]:
# Payal's parser takes HTML and yields corpus object, html_word_list

### Step 4: Visual Linking

In [ ]:
from visual_linking import link_lists

%time links = link_lists(pdf_word_list)

### Step 5: Updating with coordinates

In [2]:
# Payal's code walks through phrases, updating each one's five visual attributes
# (page, top, left, bottom, right)

### Step 6: Assess Results

In [ ]:
# Use Ines's code to plot locations of words on the original PDF